### Get sample's filename

In [1]:
import pandas as pd
import pickle

In [3]:
def get_executable_sample() -> dict:
    df = pd.read_csv("../C malware info/sample_info.csv")
    family_lst = df.family.unique()
    sample_dct = {}
    for key in family_lst:
        sample_dct[key] = []

    for idx, row in df.iterrows():
        if row.platform in ['X86_64','X86','ARM']:
            sample_dct[row.family].append(row.filename)
    return sample_dct

sample_dct = get_executable_sample()

# save dictionary
with open('sample_dct.pkl', 'wb') as outp:
    pickle.dump(sample_dct, outp, pickle.HIGHEST_PROTOCOL)

In [7]:
# load dictionary
with open('sample_dct.pkl', 'rb') as inp:
    sample_dct = pickle.load(inp)

In [5]:
sample_dct

['16b4093813e2923e9ee70b888f0d50f972ac607253b00f25e4be44993d263bd2.bin',
 '28443c0a9bfd8a12c12a2aad3cc97d2e8998a9d8825fcf3643d46012f18713f0.bin',
 '46389c117c5f41b60e10f965b3674b3b77189b504b0aeb5c2da67adf55a7129f.bin',
 '7d3855bb09f2f6111d6c71e06e1e6b06dd47b1dade49af0235b220966c2f5be3.bin']

### Build 
[solved] Must run in `.py` file, cant run in `.ipynb`. Don't know why.
- 注意 trace folder 和 csv 的家族名稱不同，一個有 `-all` 一個沒有，兩者要一致否則會出 node 都是 0。
- 跑到 xorddos 的 ./trace/Dofloo/xxx 噴 FileNotFoundError: [WinError 2] 系統找不到指定的檔案。
    - f477b05826c46ac8a482e85d89b00de0.bin (有時候又OK)

In [11]:
# ! pip install graphviz

  Using cached graphviz-0.20.1-py3-none-any.whl (47 kB)


In [11]:
from cc_module import build

for family_name in sample_dct:
    if family_name != 'Dofloo': # 只建這個 family 的圖
        continue
    print(f'building family {family_name}')
    for s in sample_dct[family_name]:
        print(f'building sample {s}')
        try:
            # 建立圖譜並存檔
            build(family_name, s)
        except FileNotFoundError:
            print('[Error] FileNotFoundError')
    # [build(family_name, s) for s in sample_dct[family_name]] #　一行搞定版

building family Dofloo
building sample 0046a78514a658b9b7a4e29e8581f85b.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 04b790e3b0dd70389c516216f81c6e5d.bin
Node before reduction:  1
1
Nodes after reduction:  1
building sample 121ca5362f9bc3fff5258b757e4250e5.bin
Node before reduction:  35
35
Nodes after reduction:  35
building sample 26d45f2e2d360ff1707c734006878dcb.bin
Node before reduction:  35
35
Nodes after reduction:  35
building sample 392214344b6d802dc4da644be260277e.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 44d98df44327179d1a11dce725d0fa4a.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 52ff8e32c390389f5bf18e0f0e2d9ada.bin
Node before reduction:  25
25
Nodes after reduction:  25
building sample 54a11aacadee0ed46e76373cfb8ed296.bin
Node before reduction:  36
36
Nodes after reduction:  36
building sample 5700e4b1c6498619a0e2562321363bd0.bin
Node before reduction:  28
28
Nodes after reduc

### Build 並回傳 ASG 資料結構
可以從 class AttackGraph 取得圖譜上的節點， asg.graph.keys()。

In [1]:
# 跑單一個圖
from cc_module import build

asg, g = build('Dofloo-all', '9a37fcc7eab08d59532bc5c66390bc30.bin')
type(g)

path: ./trace/Dofloo-all/9a37fcc7eab08d59532bc5c66390bc30.bin
Node before reduction:  36
36
Nodes after reduction:  36


graphviz.graphs.Digraph

In [8]:
names = [node.name for node in asg.graph.keys()]
set(names)

{'/dev/urandom',
 '/etc/init.d/boot.local',
 '/etc/rc.d/rc.local',
 '/etc/rc.local',
 '/proc/net/dev',
 '/proc/stat',
 '/sys/devices/system/cpu/online',
 '1529',
 '1530',
 '1531',
 '1532',
 '183.131.83.38:50050',
 'eth0',
 'eth1',
 'eth2',
 'eth3',
 'eth4',
 'eth5',
 'eth6',
 'eth7',
 'eth8',
 'eth9',
 'malware',
 'sed',
 'sh',
 'uname'}

取得所有樣本的 Node，進行比對。

In [4]:
edges = graph.edges
for nodes, edge in edges.items():
    print(nodes.name)

{(<cc_module.Node at 0x254503a3430>,
  'read'): <cc_module.Edge at 0x254503a3160>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a32b0>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3100>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3250>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3310>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a33a0>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a3130>,
 (<cc_module.Node at 0x254503a31f0>,
  'clone'): <cc_module.Edge at 0x254503a33d0>,
 (<cc_module.Node at 0x254503a3370>,
  'exec'): <cc_module.Edge at 0x254503a3610>,
 (<cc_module.Node at 0x254503a3490>,
  'read'): <cc_module.Edge at 0x254503a3190>,
 (<cc_module.Node at 0x254503a34f0>,
  'write'): <cc_module.Edge at 0x254503a3220>,
 (<cc_module.Node at 0x254503a3340>,
  'exec'): <cc_module.Edge at 0x254503a35e